In [1]:
import time
import numpy as np
from class_data import *

In [2]:
path_data = '/Users/federicogallina/Documents/PhD documents/Data/'
baci_documentation = Path(path_data + 'BACI_documentation')
baci_data = Path(path_data + 'trade_flows/BACI_HS07_V202001')
path_temp = Path('temp/')

In [3]:
country_codes = pd.read_csv(baci_documentation / "country_codes_V202001.csv", encoding='latin-1', dtype = {'country_code': 'str', 'country_name_abbreviation': 'str', 'country_name_full': 'str', 'iso_2digit_alpha': 'str', 'iso_3digit_alpha': 'str'})

In [4]:
full_kor = pd.DataFrame()
def cleaning_baci(year, country_df, empty_df, substring, col_origin, id_origin, col_destination, id_destination, input_path, output_path, root_output_file):
    df = pd.read_csv(input_path / f"BACI_HS07_Y{year}_V202001.csv")
    baci = BACI(df, country_df)
    baci.adjust_columns()
    baci.match_country_codes('origin')
    baci.match_country_codes('destination')
    baci.slice_countries(substring, {col_origin: id_origin, col_destination: id_destination})
    baci.trade_flow_class({col_origin: id_origin, col_destination: id_destination})
    baci.industry_classification()
    baci.data.to_csv(output_path / f"{root_output_file}_{year}.csv", index = False)
    empty_df = empty_df.append(baci.data, ignore_index = True)
    return empty_df

In [5]:

start = time.time()
for year in range(2007, 2019):
    full_kor = cleaning_baci(year, country_codes, full_kor, 'or', 'origin_iso3', 'KOR', 'destination_iso3', 'KOR', baci_data, path_temp, 'kor')
end = time.time()
print(end - start)

810.0483009815216


In [7]:
full_kor.to_csv(path_temp / "temp.csv", index = False)

In [5]:
full_kor = pd.read_csv(path_temp / "temp.csv")
baci_full = BACI(full_kor, country_codes)
baci_full.total_flow()
baci_full.aggregate_country(BACI.eu28, "EU28", "0")
baci_full.aggregate_country(BACI.eu27, "EU27", "1")
export_df = baci_full.data
export_df['value_log'] = np.log(export_df['value'])
export_df['value_change'] = export_df.groupby(['origin_name', 'destination_name', 'product_code'], as_index = False)['value'].diff()
export_df['value_growth'] = export_df['value_change']/(export_df['value'] - export_df['value_change'])
export_df.to_csv(path_temp / "kor_full.csv", index = False)

      year  origin  destination product_code         value      quantity  \
2302  2007      40          410        10619      1.063000      0.002000   
2303  2007      40          410        20319    420.801177    120.566275   
2304  2007      40          410        20322    469.403447    794.135463   
2305  2007      40          410        20329  51454.413816  13688.963006   
2306  2007      40          410        20629      7.892000     20.600000   

     origin_name origin_iso3 destination_name destination_iso3 trade_flow  \
2302     Austria         AUT    Rep. of Korea              KOR     Import   
2303     Austria         AUT    Rep. of Korea              KOR     Import   
2304     Austria         AUT    Rep. of Korea              KOR     Import   
2305     Austria         AUT    Rep. of Korea              KOR     Import   
2306     Austria         AUT    Rep. of Korea              KOR     Import   

     chapters   industry  
2302        1  Agri_Food  
2303        2  Agri_Food  